In [1]:
%load_ext autoreload
%autoreload 2

Load model

In [69]:
import os

from aiutils import load

In [7]:
project_root = '/home/justiny/Documents/Projects/PyDominion'
model_dir = os.path.join(project_root, 'models')

In [4]:
model_name = 'r-r-mcts-10k-score'
model_path = os.path.join(model_dir, model_name)

In [62]:
import torch

In [112]:
model = torch.load(model_path, map_location='cpu')
model

PredictorMLP(
  (fc1): Linear(in_features=21, out_features=11, bias=True)
  (fc2): Linear(in_features=11, out_features=1, bias=True)
  (sigmoid): Sigmoid()
  (relu): LeakyReLU(negative_slope=0.01)
  (tanh): Tanh()
)

In [2]:
import logging

from config import GameConfig 
from constants import BUY
from enums import StartingSplit
from env import DefaultEnvironment
from player import load_players
from simulations import simulate

In [3]:
from buyagenda import BigMoneyBuyAgenda
from player import HeuristicPlayer, PredictorMLPPlayer

In [4]:
logger = logging.getLogger()
logger.setLevel(logging.ERROR)
# logger.setLevel(BUY)

In [114]:
config = GameConfig(split=StartingSplit.StartingRandomSplit, sandbox=True, num_players=2)
players = load_players(['MLP', 'BM'], [model_path], map_location='cpu', train=False)
agent = players[0]
env = DefaultEnvironment(config, players, logger=logger)

In [117]:
simulate(env, 1, None)

  0%|          | 0/1 [00:00<?, ?it/s]BUY:root:Player 0, Turn 0: Estate
BUY:root:Player 1, Turn 1: Silver
BUY:root:Player 0, Turn 1: Estate
BUY:root:Player 1, Turn 2: Silver
BUY:root:Player 0, Turn 2: Estate
BUY:root:Player 1, Turn 3: Gold
BUY:root:Player 0, Turn 3: Estate
BUY:root:Player 1, Turn 4: Silver
BUY:root:Player 0, Turn 4: Estate
BUY:root:Player 1, Turn 5: Silver
BUY:root:Player 0, Turn 5: None
BUY:root:Player 1, Turn 6: Gold
BUY:root:Player 0, Turn 6: Estate
BUY:root:Player 1, Turn 7: Gold
BUY:root:Player 0, Turn 7: Copper
BUY:root:Player 1, Turn 8: Gold
BUY:root:Player 0, Turn 8: Estate
BUY:root:Player 1, Turn 9: Silver
BUY:root:Player 0, Turn 9: Estate
BUY:root:Player 1, Turn 10: Silver
BUY:root:Player 0, Turn 10: Copper
BUY:root:Player 1, Turn 11: Province
BUY:root:Player 0, Turn 11: Copper
BUY:root:Player 1, Turn 12: Gold
BUY:root:Player 0, Turn 12: Silver
BUY:root:Player 1, Turn 13: Province
BUY:root:Player 0, Turn 13: Copper
BUY:root:Player 1, Turn 14: Gold
BUY:root:Pla

Let's analyze the neural network outputs for some simple cases.

In [20]:
from state import ReducedStateFeature

In [21]:
skip = ReducedStateFeature.default_sandbox_feature()
copper = ReducedStateFeature.default_sandbox_feature()
silver = ReducedStateFeature.default_sandbox_feature()
estate = ReducedStateFeature.default_sandbox_feature()

In [22]:
copper[0] = 45
copper[7] = 8
copper

tensor([45., 10.,  8.,  8.,  8., 40., 30.,  8.,  0.,  3.,  0.,  0.,  0.,  0.,
         7.,  0.,  3.,  0.,  0.,  0.,  0.])

In [23]:
silver[5] = 39
silver[7+5] = 1
silver

tensor([46., 10.,  8.,  8.,  8., 39., 30.,  7.,  0.,  3.,  0.,  0.,  1.,  0.,
         7.,  0.,  3.,  0.,  0.,  0.,  0.])

In [24]:
estate[2] = 7
estate[7+2] = 4
estate

tensor([46., 10.,  7.,  8.,  8., 40., 30.,  7.,  0.,  4.,  0.,  0.,  0.,  0.,
         7.,  0.,  3.,  0.,  0.,  0.,  0.])

In [54]:
agent.model(copper), agent.model(silver), agent.model(estate)

TypeError: 'LogisticRegression' object is not callable

Tree Simulations

In [5]:
from mcts import GameTree

In [8]:
tree_name = 'r-r-mcts-10k-scaled-score'
tree_path = os.path.join(model_dir, tree_name)
rollout_name = 'mlog-10k'
rollout_path = os.path.join(model_dir, rollout_name)

In [9]:
tree = GameTree.load(tree_path, train=False)

In [18]:
config = GameConfig(split=StartingSplit.StartingRandomSplit, sandbox=True, num_players=2)
players = load_players(['UCT', 'BM'], [rollout_path], tree=tree, train=False, rollout_type='mlog')
agent = players[0]
env = DefaultEnvironment(config, players, logger=logger)

In [11]:
# logger.setLevel(logging.ERROR)
logger.setLevel(BUY)

In [19]:
simulate(env, 1, tree)

  0%|          | 0/1 [00:00<?, ?it/s]BUY:Player 0, Turn 0: Estate
BUY:Player 1, Turn 1: Silver
BUY:Player 0, Turn 1: Estate
BUY:Player 1, Turn 2: Silver
BUY:Player 0, Turn 2: Estate
BUY:Player 1, Turn 3: Silver
BUY:Player 0, Turn 3: Estate
BUY:Player 1, Turn 4: Silver
BUY:Player 0, Turn 4: None
BUY:Player 1, Turn 5: Silver
BUY:Player 0, Turn 5: Estate
BUY:Player 1, Turn 6: Gold
BUY:Player 0, Turn 6: Estate
BUY:Player 1, Turn 7: Silver
BUY:Player 0, Turn 7: Estate
BUY:Player 1, Turn 8: Gold
BUY:Player 0, Turn 8: None
BUY:Player 1, Turn 9: Province
BUY:Player 0, Turn 9: None
BUY:Player 1, Turn 10: Silver
BUY:Player 0, Turn 10: None
BUY:Player 1, Turn 11: Province
BUY:Player 0, Turn 11: Copper
BUY:Player 1, Turn 12: Gold
BUY:Player 0, Turn 12: Copper
BUY:Player 1, Turn 13: Gold
BUY:Player 0, Turn 13: Copper
BUY:Player 1, Turn 14: Silver
BUY:Player 0, Turn 14: Copper
BUY:Player 1, Turn 15: Gold
BUY:Player 0, Turn 15: Copper
BUY:Player 1, Turn 16: Province
BUY:Player 0, Turn 16: None
BUY:Pl

In [13]:
rollouts = players[0].rollout.models


In [25]:
i = 8
rollout_model = rollouts[i]
rollout_model.coef_[0][:7], rollouts[i].coef_[0][7:14], rollouts[i].coef_[0][14:]

(array([-0.19708913, -0.06241848, -0.02408548,  0.27634834,  1.24134186,
        -0.12067769,  0.11260199]),
 array([ 0.14400049,  0.        , -0.02566072,  0.91370371,  3.41859173,
         0.30908846,  0.37064732]),
 array([-0.32142226,  0.        , -0.03763968, -1.23998683, -4.70986837,
        -0.4380847 , -0.67050476]))

In [27]:
rollout_model.predict_proba([copper.numpy(), silver.numpy(), estate.numpy()])

array([[0.57142982, 0.42857018],
       [0.54958911, 0.45041089],
       [0.65256928, 0.34743072]])

In [37]:
tree.node.children

[None<--n: 832, v: 15264, c: None-->['Copper', 'Estate', 'None'],
 None<--n: 4201, v: 77413, c: None-->['Copper', 'Estate', 'Silver', 'None'],
 None<--n: 4081, v: 72140, c: None-->['Copper', 'Estate', 'Silver', 'None'],
 None<--n: 886, v: 15063, c: None-->['Copper', 'Estate', 'Duchy', 'Silver', 'None']]